In [2]:
import gradio as gr
from source.pipeline.config import PipelineConfig
from source.pipeline.controller import PipelineController
from source.pipeline.step.retrieval import RetrievalStep
from source.pipeline.step.generation import (
    GenerationStep, 
    AnswerGenerateOutputParser, 
    AnswerGeneratePromptGenerator,
    ThoughtGenerateOutputParser,
    ThoughtGeneratePromptGenerator,
)
from source.pipeline.step.end import EndStep
from source.pipeline.state import QuestionState
from source.module.generate.llama import LlamaGenerator, LlamaGeneratorConfig
from source.module.retrieve.dense import DenseRetriever, DenseRetrieverConfig
from source.module.index.index import Indexer, IndexerConfig
from source.utility.system_utils import seed_everything

from huggingface_hub import login

login(token="{Yourtoken}")

# ----------------------------
# 설정
# ----------------------------
seed_everything(100)

cfg = PipelineConfig(
    # 필요한 최소 설정
    method="base",
    batch_size=1,
    generation_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct',
    generation_max_batch_size=1,
    generation_max_total_tokens=4096,
    generation_max_new_tokens=64,
    generation_min_new_tokens=1,
    retrieval_count=8,
    retrieval_query_type='full',
    dataset='musique',
    max_num_thought=6,
    answer_regex=".* answer is:? (.*)\\.?",
)

# ----------------------------
# 모듈 초기화
# ----------------------------
generator = LlamaGenerator(
    LlamaGeneratorConfig(
        model_name=cfg.generation_model_name,
        batch_size=cfg.generation_max_batch_size,
        max_total_tokens=cfg.generation_max_total_tokens,
        max_new_tokens=cfg.generation_max_new_tokens,
        min_new_tokens=cfg.generation_min_new_tokens,
        use_vllm=False, #True,
        gpu=7,
    )
)

retriever = DenseRetriever(
    DenseRetrieverConfig(
        query_model_name_or_path='facebook/contriever-msmarco',
        passage_model_name_or_path=None,
        batch_size=32,
        training_strategy=None,
        use_fp16=False
    )
)

indexer = Indexer.load_local(
    IndexerConfig(
        embedding_sz=768,
        database_path=cfg.database_path
    )
)

# ----------------------------
# 파이프라인 구성
# ----------------------------
pipeline = [
    RetrievalStep(cfg=cfg, retriever=retriever, indexer=indexer),
    GenerationStep(cfg=cfg, generator=generator,
                   prompt_generator=AnswerGeneratePromptGenerator(cfg),
                   output_parser=AnswerGenerateOutputParser(cfg)),
    EndStep(cfg=cfg),
    GenerationStep(cfg=cfg, generator=generator,
                   prompt_generator=ThoughtGeneratePromptGenerator(cfg),
                   output_parser=ThoughtGenerateOutputParser(cfg)),
]

controller = PipelineController(
    pipeline=pipeline,
    logging_file_path=None,
    prediction_file_path=None
)

/workspace/miniconda3/envs/rescore/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-17 05:56:44,423	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Loading checkpoint shards:  75%|███████▌  | 3/4 [00:04<00:01,  1.37s/it]

In [26]:
user_input="What forced the departure from the country responsible for Phoe Pyonn Cho of the people once defied by new coinage?"

start_state = QuestionState(question_id="1", question=user_input)

In [27]:
start_state

QuestionState(state_id='06204127-4c29-48ee-bfe9-9ce454eb99fb', parent_state_id=None, question_id='1', question='Which chemical element was first discovered in a mineral found only in a single cave in Sweden in 1787 and has an atomic number between 60 and 70?', answer=None)

In [28]:
controller.update([start_state])

In [29]:
paths = controller.next()  

In [30]:
paths

[[QuestionState(state_id='06204127-4c29-48ee-bfe9-9ce454eb99fb', parent_state_id=None, question_id='1', question='Which chemical element was first discovered in a mineral found only in a single cave in Sweden in 1787 and has an atomic number between 60 and 70?', answer=None)]]

In [31]:
next_states = controller.pipeline[0](paths) 

In [32]:
next_states

[DocumentState(state_id='1f5dd1a8-232f-436c-973b-3fe0345b55f9', parent_state_id='06204127-4c29-48ee-bfe9-9ce454eb99fb', documents=[Document(id='T1J8TmJnhX7toGxXqGznu963zNpyC2xv', content='Periodic table All the elements from atomic numbers 1 (hydrogen) through 118 (oganesson) have been either discovered or synthesized, completing the first seven rows of the periodic table. The first 98 elements exist in nature, although some are found only in trace amounts and others were synthesized in laboratories before being found in nature. Elements 99 to 118 have only been synthesized in laboratories or nuclear reactors. The synthesis of elements having higher atomic numbers is currently being pursued: these elements would begin an eighth row, and theoretical work has been done to suggest possible candidates for this extension. Numerous synthetic radionuclides of naturally occurring elements have also been produced in laboratories.', metadata={'id': 'T1J8TmJnhX7toGxXqGznu963zNpyC2xv', 'title': 'P

In [33]:
controller.update(next_states)

In [34]:
paths = controller.next()

In [35]:
next_states = controller.pipeline[1](paths) 

In [36]:
next_states

[AnswerState(state_id='a5992ef8-9ee0-403b-b8fb-2571dfdc8bba', parent_state_id='1f5dd1a8-232f-436c-973b-3fe0345b55f9', question_id=None, answer='Tungsten')]

In [37]:
next_states[0].answer = "Unknown"

In [38]:
next_states[0].answer

'Unknown'

In [39]:
controller.update(next_states)

In [40]:
paths = controller.next()

In [41]:
paths

[[QuestionState(state_id='06204127-4c29-48ee-bfe9-9ce454eb99fb', parent_state_id=None, question_id='1', question='Which chemical element was first discovered in a mineral found only in a single cave in Sweden in 1787 and has an atomic number between 60 and 70?', answer=None),
  DocumentState(state_id='1f5dd1a8-232f-436c-973b-3fe0345b55f9', parent_state_id='06204127-4c29-48ee-bfe9-9ce454eb99fb', documents=[Document(id='T1J8TmJnhX7toGxXqGznu963zNpyC2xv', content='Periodic table All the elements from atomic numbers 1 (hydrogen) through 118 (oganesson) have been either discovered or synthesized, completing the first seven rows of the periodic table. The first 98 elements exist in nature, although some are found only in trace amounts and others were synthesized in laboratories before being found in nature. Elements 99 to 118 have only been synthesized in laboratories or nuclear reactors. The synthesis of elements having higher atomic numbers is currently being pursued: these elements would 

In [42]:
next_states = controller.pipeline[2](paths) 

In [43]:
next_states

[ResumeState(state_id='07a45c79-3749-47a7-b4ae-f31f02e1cc52', parent_state_id='a5992ef8-9ee0-403b-b8fb-2571dfdc8bba')]

In [44]:
controller.update(next_states)
paths = controller.next()
print(paths)

[[QuestionState(state_id='06204127-4c29-48ee-bfe9-9ce454eb99fb', parent_state_id=None, question_id='1', question='Which chemical element was first discovered in a mineral found only in a single cave in Sweden in 1787 and has an atomic number between 60 and 70?', answer=None), DocumentState(state_id='1f5dd1a8-232f-436c-973b-3fe0345b55f9', parent_state_id='06204127-4c29-48ee-bfe9-9ce454eb99fb', documents=[Document(id='T1J8TmJnhX7toGxXqGznu963zNpyC2xv', content='Periodic table All the elements from atomic numbers 1 (hydrogen) through 118 (oganesson) have been either discovered or synthesized, completing the first seven rows of the periodic table. The first 98 elements exist in nature, although some are found only in trace amounts and others were synthesized in laboratories before being found in nature. Elements 99 to 118 have only been synthesized in laboratories or nuclear reactors. The synthesis of elements having higher atomic numbers is currently being pursued: these elements would be

In [45]:
next_states = controller.pipeline[3](paths) 
print(next_states)

[ThoughtState(state_id='e2bce25f-37eb-497f-8f8b-0ae3843342b5', parent_state_id='07a45c79-3749-47a7-b4ae-f31f02e1cc52', thought='"" "The chemical element that fits the description is likely Tungsten, which was discovered by Carl Wilhelm Scheele in 1783, but it was found in a mineral in a single cave in Sweden in 1787, and its atomic number is 74, which is outside the given range,')]


In [46]:
controller.update(next_states)
paths = controller.next()

In [47]:
print(paths)

[[QuestionState(state_id='06204127-4c29-48ee-bfe9-9ce454eb99fb', parent_state_id=None, question_id='1', question='Which chemical element was first discovered in a mineral found only in a single cave in Sweden in 1787 and has an atomic number between 60 and 70?', answer=None), DocumentState(state_id='1f5dd1a8-232f-436c-973b-3fe0345b55f9', parent_state_id='06204127-4c29-48ee-bfe9-9ce454eb99fb', documents=[Document(id='T1J8TmJnhX7toGxXqGznu963zNpyC2xv', content='Periodic table All the elements from atomic numbers 1 (hydrogen) through 118 (oganesson) have been either discovered or synthesized, completing the first seven rows of the periodic table. The first 98 elements exist in nature, although some are found only in trace amounts and others were synthesized in laboratories before being found in nature. Elements 99 to 118 have only been synthesized in laboratories or nuclear reactors. The synthesis of elements having higher atomic numbers is currently being pursued: these elements would be

In [48]:
next_states = controller.pipeline[0](paths) 
print(next_states)

Carl Wilhelm Scheele is in retrieved_so_far_. Skip.
[DocumentState(state_id='d03e178e-b4c5-431e-9761-d08f484f70f1', parent_state_id='e2bce25f-37eb-497f-8f8b-0ae3843342b5', documents=[Document(id='T1J8TmJnhX7toGxXqGznu963zNpyC2xv', content='Periodic table All the elements from atomic numbers 1 (hydrogen) through 118 (oganesson) have been either discovered or synthesized, completing the first seven rows of the periodic table. The first 98 elements exist in nature, although some are found only in trace amounts and others were synthesized in laboratories before being found in nature. Elements 99 to 118 have only been synthesized in laboratories or nuclear reactors. The synthesis of elements having higher atomic numbers is currently being pursued: these elements would begin an eighth row, and theoretical work has been done to suggest possible candidates for this extension. Numerous synthetic radionuclides of naturally occurring elements have also been produced in laboratories.', metadata={'

In [50]:
controller.update(next_states)

In [51]:
paths = controller.next()

[[QuestionState(state_id='06204127-4c29-48ee-bfe9-9ce454eb99fb', parent_state_id=None, question_id='1', question='Which chemical element was first discovered in a mineral found only in a single cave in Sweden in 1787 and has an atomic number between 60 and 70?', answer=None), DocumentState(state_id='1f5dd1a8-232f-436c-973b-3fe0345b55f9', parent_state_id='06204127-4c29-48ee-bfe9-9ce454eb99fb', documents=[Document(id='T1J8TmJnhX7toGxXqGznu963zNpyC2xv', content='Periodic table All the elements from atomic numbers 1 (hydrogen) through 118 (oganesson) have been either discovered or synthesized, completing the first seven rows of the periodic table. The first 98 elements exist in nature, although some are found only in trace amounts and others were synthesized in laboratories before being found in nature. Elements 99 to 118 have only been synthesized in laboratories or nuclear reactors. The synthesis of elements having higher atomic numbers is currently being pursued: these elements would be

In [1]:
print(paths)

NameError: name 'paths' is not defined